# Generalised LR (GLR)
First introduced in 1965 by Knuth [1], LR is a bottom-up parsing technique that works by constructing an automaton, then traveling through it while consuming input one by one and maintaining a stack. A more detailed description about LR can be found [here](https://rahul.gopinath.org/post/2024/07/01/lr-parsing/).

Creating a grammar in LR(1), or even LR(k), can be difficult. Ideally, you want your grammar to be intuitive, easily understood, and readable. This is important because you'll make mistakes or change your mind in the future, and modifying a large grammar while making sure that it is in LR(1) is a painful process. Luckily, a more powerful parser, that can handle all context-free grammar, called Generalised LR (GLR) is available. This post outlines the implementation details of two GLR variants (RNGLR and BRNGLR), both of which are presented in Economopoulos's PhD dissertation [2].
### Preliminary
This post assumes that the reader is already familiar with LR parsing and context-free grammar terminology (non-terminal, derivation, nullable,...).

The parser implementation here uses the fuzzingbook format for input grammar. For example a grammar of the form $$\begin{split} S &\rightarrow A+A\ |\ A-A\\
A &\rightarrow a\ |\ b\end{split}$$
Is presented as

In [1]:
sample_grammar = {
	'<S>': [['<A>', '+', '<A>'],
			['<A>', '-', '<A>']],
	'<A>': [['a'], ['b']]
}

#### Table Generator
This GLR implementation uses LR(1) as the base parse table, here we introduce all the needed components for an LR(1) parse table generator.
##### Types and helper function

In [2]:
Grammar = dict[str, list[list[str]]]
# State in the LR automaton
State = tuple[int, set["Item"]]

LOGGING = False

def is_nt(k: str):
    '''
        Check if k is a non-terminal
    '''
    return (k[0], k[-1]) == ('<', '>')

##### Item class

In [3]:
class Item:
    def __init__(self, lhs: str, rhs: list[str], dot: int, look_ahead: str):
        '''
            An item is a production of the form A -> a·b
            
            Args:
                lhs: The left-hand side of the production (A)
                rhs: The right-hand side of the production (a b)
                dot: The position of the dot in the production (dot = 0 means A -> ·a b)
                look_ahead: look ahead symbol
        '''
        self.lhs = lhs
        self.rhs = rhs
        self.dot = dot
        self.look_ahead = look_ahead
    
    def __repr__(self):
        return f"{self.lhs} -> {' '.join(self.rhs[:self.dot])} · {' '.join(self.rhs[self.dot:])}, {self.look_ahead}"
        # format: A -> a · b, look_ahead
    
    def __eq__(self, other):
        return self.lhs == other.lhs and self.rhs == other.rhs and self.dot == other.dot and self.look_ahead == other.look_ahead

    def __hash__(self):
        return hash((self.lhs, tuple(self.rhs), self.dot, self.look_ahead))

##### TableGenerator class

In [4]:
class TableGenerator:
    def __init__(self, grammar: Grammar, start: str):
        '''
            This class is responsible for generating the right-nulled parse table.
        '''

        self.end_of_input = "€"
        self.grammar = grammar
        self.start = self.add_new_start(start)

        self.rule_list = self.reformat_grammar(grammar)
        self.nullable = TableGenerator.get_nullable(grammar)
        self.first = self.init_FIRST(grammar)
        self.follow = self.init_FOLLOW(grammar, self.start)

        self.symbols = self.get_symbols(grammar)

        self.sppf = SPPF(grammar)
    
    def get_symbols(self, grammar: Grammar):
        '''
            Return the list of all terminals, and non-terminals
        '''
        symbols: set[str] = set()
        for nt in grammar:
            for production in grammar[nt]:
                for symbol in production:
                    symbols.add(symbol)
        
        return symbols
    
    def add_new_start(self, start:str):
        '''
            Augment new start symbol <S#>
        '''
        new_start = ''.join([start[:-1], "#", start[-1:]])
        new_production = [[start]]
        self.grammar[new_start] = new_production

        return new_start

    def reformat_grammar(self, grammar: Grammar) -> list[tuple[str, list[str]]]:
        '''
            Transform grammar from a dictionary to a list of rules

            args
                grammar: the given grammar, in dictionary form
            
            return
                A list of rules, a rule is a tuple with lhs and rhs 
        '''

        rule_list = []

        for nt in grammar.keys():
            for production in grammar[nt]:
                rule_list.append((nt, production))
        return rule_list
    
    @staticmethod
    def get_nullable(grammar: Grammar) -> set[str]:
        '''
            Get nullable set, a non-terminal is called nullable if X can derive epsilon 

            return
                The set of nullable non-terminals
        '''
        res: set[str] = set()
        prev_size = -1
        while (prev_size != len(res)):
            prev_size = len(res)
            for nt in grammar.keys():
                for production in grammar[nt]:
                    if (len(production) == 0):
                        res.add(nt)
                        continue
                    
                    if (all((is_nt(symbol) and symbol in res) for symbol in production)):
                        res.add(nt)
        
        return res
    
    def init_FIRST(self, grammar: Grammar) -> dict[str, set[str]]:
        '''
            Calculate the FIRST set for all symbols
            FIRST(X) is the set of terminals that can start a string derived from X

            return
                A dictionary with all the non-terminal as keys and FIRST sets as values
        '''
        first: dict[str, set[str]] = {}
        for nt in grammar:
            first[nt] = set()
            if nt in self.nullable:
                first[nt].add("epsilon")
        
        changed = True
        while (changed):
            changed = False
            for lhs in grammar:
                for rhs in grammar[lhs]:
                    # Epsilon rule, already handled above
                    if (len(rhs) == 0):
                        continue

                    for symbol in rhs:
                        if (is_nt(symbol)):
                            for char in first[symbol]:
                                if char not in first[lhs]:
                                    first[lhs].add(char)
                                    changed = True
                            
                            if symbol not in self.nullable:
                                break
                        # Is terminal
                        else:
                            if symbol not in first[lhs]:
                                first[lhs].add(symbol)
                                changed = True
                            break
    
        return first
    
    def calculate_FIRST(self, symbol_list: list[str]) -> set[str]:
        '''
            Calculate FIRST(X), where X is one or more non-terminals/terminals sequence

            args
                symbol_list: list of symbols
            return
                The FIRST set
        '''

        if (len(symbol_list) == 0):
            return {"epsilon"}

        res = set()
        nullable = True
        for symbol in symbol_list:
            if is_nt(symbol):
                for char in self.first[symbol]:
                    if char != "epsilon":
                        res.add(char)
                
                if symbol not in self.nullable:
                    nullable = False
                    break
            else:
                nullable = False
                res.add(symbol)
                break

        if nullable:
            res.add("epsilon")
        
        return res

    
    def init_FOLLOW(self, grammar: Grammar, start: str) -> dict[str, set[str]]:
        '''
            Calculate the FOLLOW set for all symbols
            FOLLOW(A) is the set of terminals that can appear immediate after A
            Example: S -> A a B c then a is in FOLLOW(A)

            return
                A dictionary with all the non-terminal as keys and FOLLOW sets as values
        '''
        follow: dict[str, set[str]] = {}
        for nt in grammar:
            follow[nt] = set()
        follow[start].add(self.end_of_input)

        changed = True
        while (changed):
            changed = False
            for lhs in grammar:
                for rhs in grammar[lhs]:
                    for idx, symbol in enumerate(rhs):
                        if not is_nt(symbol):
                            continue
                        
                        # lhs -> ...By
                        # Adding FIRST(y) to FOLLOW(B)
                        # print(symbol + " " + str(rhs[idx + 1:]))
                        first_y = self.calculate_FIRST(rhs[idx + 1:])
                        for char in first_y:
                            if char == "epsilon":
                                continue
                            if char not in follow[symbol]:
                                changed = True
                                follow[symbol].add(char)
                        
                        if "epsilon" in first_y:
                            # Adding all symbol from FOLLOW(lhs) to FOLLOW(B)
                            for char in follow[lhs]:
                                if char not in follow[symbol]:
                                    changed = True
                                    follow[symbol].add(char)
                        
        return follow

    def find_closure(self, items: list[Item]) -> set[Item]:
        '''
            Find the closure of a list of items

            args
                items: item list

            return
                The closure set of input items
        '''
        res = set(items)
        prev_len = -1
        while (prev_len != len(res)):
            prev_len = len(res)
            for item in res.copy():
                # dot at the end
                if (item.dot == len(item.rhs)):
                    continue

                next_sym = item.rhs[item.dot]
                if not is_nt(next_sym):
                    continue
                
                # X -> α·Yβ, a
                # Calculate FIRST(βa)
                first_set = self.calculate_FIRST(item.rhs[item.dot + 1:] + [item.look_ahead])
                
                for production in self.grammar[next_sym]:
                    for look_ahead in first_set:
                        res.add(Item(next_sym, production, 0, look_ahead))
        return res
    
    def transition(self, state: list[Item], next_sym: str) -> set[Item]:
        '''
            Calculate the transition from a state with <next_sym> edge

            arg
                state: current state
                next_sym: transition edge, can be terminal or non-terminal
            return
                Next state, with all items in a set
        '''        
        items = []
        for item in state:
            if item.dot + 1 > len(item.rhs):
                continue
            if (item.rhs[item.dot] == next_sym):
                new_item = copy(item)
                new_item.dot = new_item.dot + 1
                items.append(new_item)
        
        return self.find_closure(items)
    
    def generate_states(self) -> tuple[list[State], dict[tuple[int, str], int]]:
        '''
            This function generates all the states needed for the automata
                - State format: a tuple (state_id, set of Items)

            return
                A tuple consists of
                - A list of states
                - A GOTO map: a dictionary, with (id, symbol) as keys and next_id as values
                    Example: state_1 --A--> state_2, then GOTO[(1, "A")] = 2
        '''
        # Initial state has [<S#> -> ·<S>, $] and its closure
        initial_state = (0, self.find_closure([Item(self.start, 
                                                    self.grammar[self.start][0], 
                                                    0, 
                                                    self.end_of_input)]))
        states: list[State] = []
        states.append(initial_state)
        unprocessed_states = [initial_state]

        goto_map: dict[tuple[int, str], int] = {}
        while (len(unprocessed_states) > 0):
            top_state_id, top_state = unprocessed_states.pop()
            for item in top_state:
                if (item.dot == len(item.rhs)):
                    continue

                next_sym = item.rhs[item.dot]
                next_state = self.transition(top_state, next_sym)

                # Check if state already exist
                duplicate = False
                dup_idx = 0
                for idx, state in states:
                    if (all([(item in state) for item in next_state]) and 
                        len(next_state) == len(state)):
                        dup_idx = idx
                        duplicate = True
                        break

                if not duplicate:
                    new_state = (len(states), next_state)
                    states.append(new_state)
                    unprocessed_states.append(new_state)
                    goto_map[(top_state_id, next_sym)] = new_state[0]
                else:
                    goto_map[(top_state_id, next_sym)] = dup_idx

        # Print result
        if (LOGGING):
            for state in states:
                print(f"State {state[0]}")
                for ins in state[1]:
                    print(ins)
                print("")
            
            print("---------------")
            print("Transition map")
            for key, value in goto_map.items():
                print(f"GOTO {key} = {value}")
        
        return (states, goto_map)

	def export_to_csv(table, path: str):
        # table = self
        row_entries = list(table.keys())
        symbols = list(table[0].keys())
        header = ["state"] + list(table[0].keys())

        with open(path, 'w') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            writer.writerow(header)
            for state_id in row_entries:
                row = [str(state_id)]
                for symbol in symbols:
                    row.append('/'.join(table[state_id][symbol]))
                writer.writerow(row)

## Extending LR Parser
### Eliminating Nondeterminism
If you are familiar with LR, you probably know about *shift/reduce conflicts* (choices between shifting and reducing) and *reduce/reduce conflicts* (choices between reducing different rules), a normal LR parser cannot handle conflicts, as it does not know which choice to make. What we can do is incorporating a bit of breadth-first search, so the parser can try all options, and that is the main idea behind GLR.

For example, consider the following ambiguous grammar:
$$\begin{split} S &\rightarrow a \ B \ c & \hspace{1cm} (1)\\
S &\rightarrow a\ D \ c &\hspace{1cm} (2) \\
B &\rightarrow b &\hspace{1cm} (3) \\
D & \rightarrow b &\hspace{1cm} (4)\end{split}$$
For this grammar, the LR(1) automaton is as below:
![sss](images/lr1_gram.png)
And the LR(1) parse table is:

| state | a   | b   | c               | $       | S   | B   | D   |
| ----- | --- | --- | --------------- | ------- | --- | --- | --- |
| 0     | p2  |     |                 |         | p1  |     |     |
| 1     |     |     |                 | acc     |     |     |     |
| 2     |     | p4  |                 |         |     | p5  | p3  |
| 3     |     |     | p7              |         |     |     |     |
| 4     |     |     | r(B, 3)/r(D, 4) |         |     |     |     |
| 5     |     |     | p6              |         |     |     |     |
| 6     |     |     |                 | r(S, 1) |     |     |     |
| 7     |     |     |                 | r(S, 2) |     |     |     |

In this table, "$pk$" is shift action, it means "go to state $k$" and $r(X, m)$ is the reduce action meaning "reduce symbol $X$ with rule numbered $m$." The symbol $\$$ is used to denote "end of string." There is a reduce/reduce conflict in state 4. Let's see what happens when we try to parse the string "$abc$".

| Step | Input | State | Stack                     | Next operation    |
| ---- | ----- | ----- | ------------------------- | ----------------- |
| 0    | ""    | 0     | $\$, S_0$                 | $p2$              |
| 1    | "a"   | 2     | $\$, S_0, a, S_2$         | $p4$              |
| 2    | "ab"  | 4     | $\$, S_0, a, S_1, b, S_4$ | $r(B, 3)/r(D, 4)$ |

A usual LR parser now has to choose between two possible reductions ($B \rightarrow b$ and $D \rightarrow b$). With GLR, it can attempt to try all options, but how would it do that? The simplest solution is to duplicate the stack and treat each stack as a separate process. After performing $r(B, 3)$ the stack is $\{\$, S_0, a, S_1, B, S_5\}$; similarly, we obtain $\{\$, S_0, a, S_1, D, S_3\}$ when $r(D, 4)$ is applied. Now we have 2 different stacks to manage, and the parse can continue to process with both stacks. However, this approach is not ideal, the number of stacks can blow up exponentially, we need something more efficient.
### Graph-Structured Stack (GSS)
In the above example, notice that the first four elements are the same in both stacks, therefore we can "share" them in a unified data structure. This is a "Graph-Structured Stack", or GSS, proposed by Tomita in his book [2]. 
![ddd](images/GSS_2.png)

This image illustrates how the states $S_0$ and $S_1$ are shared between the two stacks. As the name suggests, our stack is now a single graph, and each element in the stack is a node. In the original Tomita's approach, elements $a$, $D$ and $B$ are individual nodes, but here we have simplified by making them the edge labels between states.

The nodes are divided into $n+1$ *levels,* with $n$ as the length of the input string. GSS construction is done level by level, and a new level is created upon a *shift* action. The GSSNode data structure is as follow:

In [5]:
class GSSNode:
    '''
        Represent a node in the GSS structure, nodes are identified by id
    '''
    def __init__(self, level: int, id: int, label):
        self.level = level
        self.id = id
        self.label = label
        self.children: list[tuple['GSSNode', 'SPPFNode']] = []

    def __repr__(self):
        repr = f"Node({self.label})"
        return repr

    def __eq__(self, other):
        return self.id == other.id

    def __hash__(self):
        return hash(self.id)

    def add_child(self, child: 'GSSNode', edge):
        self.children.append((child, edge))

The GSS is a bit unusual. It does not perform the "pop" operation like an ordinary stack. Once a node is created, it is never removed. Instead of popping $m$ nodes out of the stack, we perform a traversal of length $m$ from the original node. For example, instead of popping 2 elements from node $S_3$, we traverse down the graph with length 2 and find that node $S_0$ is our target. We define a method to perform this operation:

In [6]:
class GSSNode(GSSNode):
    def find_paths_with_length(self, m: int) -> set[tuple['GSSNode',...]]:
        '''
            Find a set of nodes with length m from the origin node,
            return tuples of lenght m in a set, tuples contain all the labels and the destination node
        '''
        
        res: set[tuple] = set()
        def dfs(node: GSSNode, path: list[GSSNode]):
            if (len(path) >= m):
                res.add(tuple(path + [node]))
                return

            for child, edge in node.children:
                dfs(child, path + [edge])

        dfs(self, [])
        return res

The `find_paths_with_length()` method doesn't have to account for cycle because GSS is a directed acyclic graph, a cycle cannot exist. Finally we can have our GSS class:

In [7]:
class GSS:
    '''
        A Graph Structured Stack (GSS)
    '''
    def __init__(self):
        '''
            Initialize the graph, in RNGLR, a GSS has n levels, where n is the length of input string

            Each level is a set of GSSNodes, levels are stored in a list
        '''
        self.level: list[set[GSSNode]] = []
        self.count = 0

    def resize(self, n: int):
        '''
            Resize the GSS to include n levels
        '''
        self.level = [set() for i in range(n)]

    def create_node(self, label, level: int):
        '''
            Create a new node with label in a specific level
        '''
        new_node = GSSNode(level, self.count, label)
        self.count += 1
        self.level[level].add(new_node)
        return new_node
    
    def find_node(self, label, level: int) -> GSSNode:
        '''
            Find a node with label and in a specific level

            return
                GSSNode object if found, else None is returned
        '''
        # Can be optimized further
        for node in self.level[level]:
            if (node.label == label):
                return node
        return None
    
    def __repr__(self):
        '''
            Print the GSS structure
        '''
        repr = "GSS:\n"
        for idx, level in enumerate(self.level):
            repr += f"Level {idx}:\n"
            for node in level:
                repr += f"    {node}\n"
                for child, edge in node.children:
                    repr += f"        {child} - {edge}\n"
        return repr

[Maybe have a parse example with GSS here]
### Shared Packed Parse Forest (SPPF)
For practical usage, we are more interested in a full parser rather than just a recogniser. While a recogniser's output is simply a yes/no answer, a parser has to provide a full derivation path (usually in the form of the parse tree). However, a parse tree is insufficient because we are dealing with all context-free grammars, which includes ambiguous grammars; thus, multiple derivations (or even infinite ones) are possible. Instead of a parse tree, a data structure called *Shared Packed Parse Forest* (SPPF) is used.

Consider the string "abc" in the above example, we have 2 possible derivations, resulting in 2 parse trees: 
![parse tree](images/Parse_tree.png)
In an SPPF, we combine them into a single graph, the final result looks like this
![sppf](images/SPPF.png)

Nodes like "S", "a", "b" and "c" are shared to reduce space. Since $S$ can be derived in two ways (either $S\rightarrow a\ B\ c$ or $S\rightarrow a\ D\ c$), two new black nodes are created to represent different choices. These are called **packing nodes**.

In [12]:
class PackingNode:
    def __init__(self):
        self.edges = []

    def add_edge(self, node):
        self.edges.append(node)

    def __repr__(self):
        return f"PackingNode({self.edges})"

In the RNGLR algorithm, SPPF nodes are identified by (label, start position), we will discuss more about start position later.

In [13]:
class SPPFNode:
    def __init__(self, id: int, label: str, start_pos:int = -1):
        '''
            start_pos = -1 means the node is in epsilon-SPPF
        '''
        self.id = id
        self.label = label
        self.start_pos = start_pos
        self.children: list['SPPFNode' | PackingNode] = []
    
    def add_child(self, node):
        self.children.append(node)
    
    def check_sequence_exists(self, nodes: list['SPPFNode']) -> bool:
        '''
            Check if a sequence of nodes already exists in the current node
        '''
        
        # If packing nodes exist
        if any(isinstance(child, PackingNode) for child in self.children):
            for child in self.children:
                if child.edges == nodes:
                    return True
            return False
        
        # No packing nodes case
        return self.children == nodes
    
    def add_children(self, nodes: list['SPPFNode']):
        '''
            Add a list of nodes to the current node
        '''
        if len(self.children) == 0:
            for node in nodes:
                self.add_child(node)
            return
        
        # If already exists, we skip
        if self.check_sequence_exists(nodes):
            return
        
        # No packing node yet
        if not isinstance(self.children[0], PackingNode):
            z = PackingNode()
            for child in self.children:
                z.add_edge(child)
            self.children = [z]
        
        t = PackingNode()
        for node in nodes:
            t.add_edge(node)
        self.children.append(t)
    
    # Nodes are identified by (label, start_pos)
    def __hash__(self):
        return hash((self.label, self.start_pos))

    def __eq__(self, other):
        return (self.label == other.label and self.start_pos == other.start_pos)

    def __repr__(self):
        return f"SPPF Node:({self.label}, {self.start_pos})"

For the `add_children()` method, it tries to maintain the following property for every node in the SPPF:
- Each choice is unique, there is no overlapping choice.
- If there is only one possible choice, no *packing node* is used.
- If there are at least 2 choices, all choices must be wrapped in *packing nodes*.

And finally we have the SPPF class:

In [14]:
class SPPF:
    def __init__(self, grammar: Grammar):
        self.grammar = grammar

        # Two dictionary node_id -> Node and node_label -> node_id
        self.epsilon_sppf, self.I = self.build_epsilon_sppf()

        self.nodes: list[SPPFNode] = []
        self.counter = 0
    
    def create_node(self, label: str, start_pos: int) -> SPPFNode:
        node = SPPFNode(self.counter, label, start_pos)
        self.counter += 1
        self.nodes.append(node)
        return node

    def __repr__(self):
        repr = "SPPF:\n"
        for node in self.nodes:
            repr += f"    {node.label}-{node.start_pos}\n"
            for child in node.children:
                if isinstance(child, PackingNode):
                    repr += f"        PackingNode\n"
                    for edge in child.edges:
                        repr += f"            {edge}\n"
                else: repr += f"        {child}\n"
        return repr

#### Epsilon SPPF
An SPPF tree for a nullable string or symbol is called *epsilon-SPPF* (or $\epsilon$-SPPF). We precompute $\epsilon$-SPPF trees for nullable non-terminals ($A \overset{*}\rightarrow \epsilon$), this step is necessary for our parser later. In addition to non-terminals, we also build an $\epsilon$-SPPF tree for every string $\beta$ such that $\beta\overset{*}\rightarrow \epsilon$ and there exists a rule $A \rightarrow \alpha \beta$ ($\alpha \neq \epsilon$) in the grammar, such string $\beta$ is also called *right-nullable*. Finally, we define $I$ as an index function which accepts a non-terminal/string and return the corresponding $\epsilon$-SPPF root.

Let's look at an example, grammar 5.3 in the dissertation:
$$\begin{split} S &\rightarrow a \ B \ B \ C\\
B &\rightarrow b \ |\ \epsilon \\
C & \rightarrow \epsilon\end{split}$$
We have $B$ and $C$ as nullable non-terminals, and the strings $BBC$ and $BC$ satisfy the conditions for $\beta$. Therefore we build the $\epsilon$-SPPF for $B$, $C$, $BB$ and $BBC$:

![epsilonSPPF](images/epsilonSPPF.png)

In this tree, vertices are indexed from 1 to 4, hence, our $I$ function can be defined with $I(B) = 1$, $I(C) = 2$, $I(BC)=3$ and $I(BBC)=4$.

In [15]:
class SPPF(SPPF):
	def build_epsilon_sppf(self) -> tuple[dict[int, SPPFNode], dict[str, int]]:
	        '''
	            Build an epsilon-SPPF tree
	
	            return
	                A tuple that contains
	                - All SPPFNodes created, stored in a dict
	                - The I function dictionary
	        '''
	        # key: node_id, value: SPPF Node
	        epsilon_sppf: dict[int, SPPFNode] = {}
	
	        # Create epsilon node
	        eps_node = SPPFNode(0, "epsilon")
	        epsilon_sppf[0] = eps_node
	        counter = 1
	
	        # Find a given node with label
	        node_with_label: dict[str, SPPFNode] = {}
	
	        nullable = RNGLRTableGenerator.get_nullable(self.grammar)
	
	        # Step 1, add all nullable symbols
	        # Sorted to guarantee determinism
	        for nt in sorted(nullable):
	            node = SPPFNode(counter, nt)
	            epsilon_sppf[counter] = node
	            node_with_label[nt] = node
	            counter += 1
	        
	        for lhs in self.grammar:
	            for rhs in self.grammar[lhs]:
	                # Epsilon rule
	                if len(rhs) == 0:
	                    node_with_label[lhs].add_child(eps_node)
	                # Total nullable
	                elif all(x in nullable for x in rhs):
	                    node = PackingNode()
	                    for nt in rhs:
	                        node.add_edge(node_with_label[nt])
	                    node_with_label[lhs].add_child(node)
	                # Partial nullable
	                else:
	                    for i in range(1, len(rhs)):
	                        partial_rhs = rhs[i:]
	                        if len(partial_rhs) == 0:
	                            continue
	
	                        if all(x in nullable for x in partial_rhs):
	                            label = ''.join(partial_rhs)
	                            if label in node_with_label:
	                                continue
	                            node = SPPFNode(counter, label)
	                            for x in partial_rhs:
	                                node.add_child(node_with_label[x])
	                            node_with_label[label] = node
	                            epsilon_sppf[counter] = node
	                            counter += 1
	
	        # Construct the I indexing map label -> node_id
	        I: dict[str, int] = {}
	        for label, node in node_with_label.items():
	            I[label] = node.id
	        
	        return (epsilon_sppf, I)

### Right-Nulled GLR (RNGLR)
In Tomita's book, he introduced 4 different algorithms. The first one only works for grammar without $\epsilon$-rules. Algorithm 2 and 3 were intended to handle $\epsilon$-rules but failed to deal with hidden left-recursion in grammars. Algorithm 4 (which is the full parser) inherited the same problem from algorithm 2 and 3. RNGLR is an extension to algorithm 1 to include grammars with $\epsilon$-rules. 
#### Right-nulled parse table
Our algorithm uses a slightly modified parse table, which is neither LR(1) nor LALR(1). This table is built upon the usual LR table, but with the addition of new reductions for "*right-nullable*" rules; therefore it is called *right-nulled parse table*. A *right-nullable* rule has the form $A\rightarrow \alpha\beta$, where $\beta$ can derive to $\epsilon$. If an reduction *item* is of the form ($A\rightarrow \alpha \cdot\beta, a$), we write $r(A, m, f)$ into the parse table, with $m=|\alpha|$ and $f=I(\beta)$ if $m\neq0$ and $f=I(A)$ if $m=0$.

Back to grammar 5.3 
$$\begin{split} S &\rightarrow a \ B \ B \ C\\
B &\rightarrow b \ |\ \epsilon \\
C & \rightarrow \epsilon\end{split}$$
![automata](images/grammar_53_automata.png)

The regular LR(1) parse table for this grammar is 

| State | B   | C   | S   | a   | b             | $          |
| ----- | --- | --- | --- | --- | ------------- | ---------- |
| 0     |     |     | p2  | p1  |               |            |
| 1     | p4  |     |     |     | p3/r(B, 0, 1) | r(B, 0, 1) |
| 2     |     |     |     |     |               | acc        |
| 3     |     |     |     |     | r(B, 1, 0)    | r(B, 1, 0) |
| 4     | p5  |     |     |     | p6            | r(B, 0, 1) |
| 5     |     | p7  |     |     |               | r(C, 0, 2) |
| 6     |     |     |     |     |               | r(B, 1, 0) |
| 7     |     |     |     |     |               | r(S, 4, 0) |

At cell $T(5, \$)$, we can see that the parser is performing the reduction $C \rightarrow \epsilon$, in this case $m = |\alpha| = 0$ and $I(C) = 2$, hence we write $r(C, 0, 2)$. To form a *right-nulled* parse table, we need to add more reductions for right-nullable items. As the strings $BBC$ and $BC$ are nullable, such items in this case are $S\rightarrow a\cdot B\ B\ C$, $S\rightarrow a\ B\cdot B\ C$ and $S\rightarrow a\ B\ B \cdot C$. Three new reductions are added into the table:

| State | B   | C   | S   | a   | b             | $                      |
| ----- | --- | --- | --- | --- | ------------- | ---------------------- |
| 0     |     |     | p2  | p1  |               |                        |
| 1     | p4  |     |     |     | p3/r(B, 0, 1) | r(B, 0, 1) /r(S, 1, 4) |
| 2     |     |     |     |     |               | acc                    |
| 3     |     |     |     |     | r(B, 1, 0)    | r(B, 1, 0)             |
| 4     | p5  |     |     |     | p6            | r(B, 0, 1) /r(S,2,3)   |
| 5     |     | p7  |     |     |               | r(C, 0, 2)/r(S, 3, 2)  |
| 6     |     |     |     |     |               | r(B, 1, 0)             |
| 7     |     |     |     |     |               | r(S, 4, 0)             |

We have the generate parse table method:

In [ ]:
class TableGenerator(TableGenerator):
	def generate_parse_table(self):
        '''
            This function generates an LR(1) parse table for the given grammar

            return
                The parse table in the form of a 2-dimensional dictionary.
                Usage: T[\<state_id\>][\<symbol\>], each item is a list of possible action either "pk" or "r(A, p)"
        '''
        states, goto_map = self.generate_states()

        row_entries = [state[0] for state in states]
        column_entries = (list(self.symbols) + [self.end_of_input])
        column_entries.sort()
        
        table: dict[int, dict[str, list[str]]] = {}
        # Init table
        for row in row_entries:
            table[row] = {}
            for col in column_entries:
                table[row][col] = []

        # Add shift and goto
        for state, symbol in goto_map.keys():
            table[state][symbol].append(f"p{goto_map[(state, symbol)]}")

        # Add reduce
        for state_id, state in states:
            for item in state:
                # Dot at the end
                if (item.dot == len(item.rhs)):
                    if item.lhs == self.start:
                        table[state_id][self.end_of_input].append("acc")
                    else:
                        action = f"r{item.lhs}{item.dot}.{0}"
                        if (item.dot == 0):
                            action = f"r{item.lhs}{item.dot}.{self.sppf.I[item.lhs]}"
                        table[state_id][item.look_ahead].append(action)
                # Right-nulled
                else:
                    right_seq = item.rhs[item.dot:]
                    if (all([sym in self.nullable for sym in right_seq])):
                        label = ''.join(right_seq)
                        action = ""
                        if item.lhs == self.start:
                            action = "acc"
                        else:
                            action = f"r{item.lhs}{item.dot}.{self.sppf.I[label]}"
                        table[state_id][item.look_ahead].append(action)

        # print table
        if (LOGGING):
            print("\nParsing table:\n")
            frmt = "{:>12}" * len(column_entries)
            print(" ", frmt.format(*column_entries), "\n")
            ptr = 0
            for state_id in row_entries:
                print(f"{{:>3}}".format('I'+str(state_id)), end="")
                for symbol in column_entries:
                    list_opp = []
                    for opp in table[state_id][symbol]:
                        word = ""
                        word += opp
                        list_opp.append(word)
                    print(f"{{:>12}}".format("/".join(list_opp)), end="")
                print()
        return table

#### The RNGLR parser
With GSS, SPPF and the parse table ready, we are now ready to build the RNGLR parser. The parser processes input string one by one, for each symbol a new GSS level is created, it then performs every possible reduction before shifting. Reduction and shift are performed by $\mathrm{Reducer}$ and $\mathrm{Shifter}$ respectively. Two special bookkeeping sets $\mathcal{Q}$ and $\mathcal{R}$ are used to store pending shift and reduction actions. In general:
- $\mathcal{Q}$ stores the shift actions in the form of $(v, k)$, which means "from node $v$ shift into the direction of $k$" where $k$ is a terminal and $v$ is a node in GSS. Elements in $\mathcal{Q}$ are processed by the $\textrm{Shifter}$.
- $\mathcal{R}$ stores the reduction actions. Whenever a new edge between $v$ and $w$ is created in the GSS, all applicable reductions from $v$ are processed. For a reduction $r(X, m, f)$, we add $(w, X, m, f, z)$ into $\mathcal{R}$ where $z$ is the SPPF node that between $v$ and $w$, if $m = 0$ then $z$ is the $\epsilon$ node.

In addition to $\mathcal{Q}$ and $\mathcal{R}$, a set $\mathcal{N}$ is also used to bookkeep SPPF nodes, set $\mathcal{N}$ is reset after each parser iteration.

In [ ]:
class RNGLRParser:
    '''
        The RNGLR parser
    '''
    def __init__(self, start: str, grammar: Grammar, table: dict[int, dict[str, list[str]]]):
        '''
            Initialize RNGLR
        '''
        self.start = self.augment_start(start)
        self.grammar = grammar
        self.table = table

        self.input_str = ""
        self.end_of_input = "€"
        self.gss = GSS()

        # R and Q set, respectively
        self.reductions: list[tuple[GSSNode, str, int]] = []
        self.shifts: list[tuple[GSSNode, int]] = []

        self.accept_states: set[int] = self.get_accept_states()
        
        self.sppf = SPPF(grammar)
        self.set_N: dict[tuple[str, int], SPPFNode] = {}

	def augment_start(self, start: str):
        '''
            Reformat the start symbol to <S#>
        '''
        new_start = ''.join([start[:-1], "#", start[-1:]])
        return new_start

    def get_accept_states(self) -> set[int]:
        '''
            Get the accept states from the parsing table
        '''
        ret = set()
        for state_id in self.table:
            if self.end_of_input in self.table[state_id]:
                for action in self.table[state_id][self.end_of_input]:
                    if action == "acc":
                        ret.add(state_id)
        return ret

    @staticmethod
    def get_action(action: str) -> tuple[str, ...]:
        '''
            Parse the action string
            
            args
                action: the action string, either "pk" or "r\<A\>k"
            
            return
                - pk -> ("p", k)
                - r<A>m.f -> ("r", "\<A\>", m, f)
                - m, f are integers
        '''
        if (action == "acc"):
            return ("acc",)
        action_char = action[0]
        assert(action_char == 'p' or action_char == 'r')

        if (action_char == 'p'):
            number = int(action[1:])
            return (action_char, number)

        # Case "r"
        first_idx = action.find("<")
        last_idx = action.rfind(">")
        symbol = action[first_idx:last_idx + 1]
        dot_separator = action.rfind(".")
        number_1 = int(action[last_idx + 1:dot_separator])
        number_2 = int(action[dot_separator + 1:])
        return (action_char, symbol, number_1, number_2)

##### Parser pseudocode
$U_i$ is level $i$ in the GSS 
**Input:** string $a_0a_1\dots a_{n-1}$, start state $S_S$, accept state $S_A$, table $T$ 
**Parse($S$)**
- If $n$ is 0
	- If $acc \in T(S_S, \$)$
		- return success
	- return failure
- Else
	- Initialisation
	- Look at $T(S_S, a_1)$
		- Add all applicable shift actions to $\mathcal{Q}$
		- Add all applicable reduce actions to $\mathcal{R}$
	- For $i = 0$ to $n$ do
		- If $U_i$ is not empty
			- $\mathcal{N} \gets \emptyset$
			- While $\mathcal{R} \ne \emptyset$
				- $\textrm{Reducer}(i)$
			- $\textrm{Shifter(i)}$
	- If $S_A \in U_n$
		- set SPPF root
		- return success
	- return failure

In [ ]:
class RNGLRParser(RNGLRParser):
	def parse(self, input_str: str):
	        '''
	            The RNGLR recongizer, implemented based on pseudocode by Giorgios Robert Economopoulos
	        '''
	        sppf_root = None
	        result = False
	        try:
	            if (len(input_str) == 0):
	                if "acc" in self.table[0][self.end_of_input]:
	                    sppf_root = self.sppf.epsilon_sppf[self.sppf.I[self.start]]
	                    result = True
	            else:
	                # Init step
	                input_str = input_str + self.end_of_input
	                self.input_str = input_str
	                self.gss.resize(len(input_str))
	                v_0 = self.gss.create_node(0, 0)
	
	                # Check T(S, a_0)
	                for action in self.table[0][input_str[0]]:
	                    action = RNGLRParser.get_action(action)
	                    if (action[0] == 'p'):
	                        self.shifts.append((v_0, action[1]))
	                    if (action[0] == 'r'):
	                        # Reduce
	                        if (action[2] == 0):
	                            # Add (v_0, X, 0, f, epsilon)
	                            self.add_reduction(v_0, action[1], 0, action[2], self.sppf.epsilon_sppf[0])
	
	                # Now we parse
	                for i in range(len(input_str)):
	                    if len(self.gss.level[i]) > 0:
	                        self.set_N = {}
	                        while len(self.reductions) > 0:
	                            self.reducer(i)
	                        self.shifter(i)
	
	                        # if len(self.reductions) == 0:
	                        #     break
	
	                # Check accept state
	                for state in self.accept_states:
	                    acc_node = self.gss.find_node(state, len(input_str) - 1)
	                    if acc_node is not None:
	                        result = True
	
	                        # Find SPPF root
	                        for child in acc_node.children:
	                            if child[0].label == v_0.label:
	                                sppf_root = child[1]
	                        
	                        # print(f"SPPF root: {sppf_root}")
	                        break
	        
	        except KeyError as e:
	            result = False
	        
	        # print(self.gss)
	        return (result, sppf_root)

##### Reducer pseudocode
**Reducer($i$)**
- Get top element $(v, X, m, f, y)$ from $\mathcal{R}$
- Find all the paths in the GSS with length $\max(m - 1, 0)$ starting from $v$, call the set of paths $\mathcal{X}$
- For $path$ in $\mathcal{X}$
	- Let $u$ be the final node in $path$
	- if $m = 0$
		- $z\gets$ node $f$ in the $\epsilon$-SPPF tree
	- else
		- Let $c$ be the level of $u$ in GSS
		- Find SPPF node $z = (X, c)$ in $\mathcal{N}$
			- If not exist then create $z$ and add to $\mathcal{N}$
	- Let $k$ be the label of $u$ and $pl$ be the shift action in $T(k, a_i)$
	- If exists node $w$ with label $l$ in GSS level $i$
		- If edge $(w, u)$ does not exist
			- Create edge $(w, u)$
			- For $r(B, t, f) \in T(l, a_i)$
				- If $t \neq 0$ add $(u, B, t, f, z)$ to $\mathcal{R}$
	- Else
		- Create node $w$
		- Create edge $(w, u)$
		- For action in $T(l, a_i)$
			- If shift action $ph$
				- Add $(w, h)$ to $\mathcal{Q}$
			- If reduce action $r(B, t, f)$
				- If $t = 0$
					- Add $(w, B, t, f, \epsilon)$ to $\mathcal{R}$
				- else if $m\neq 0$
					- Add $(w, B, t, f, z)$ to $\mathcal{R}$
	- If $m\neq 0$
		- $nodeSequence \gets$ $w_{m-1},\dots,w_1$ be the edge labels on the path
		- Append $y$ to $nodeSequence$
		- If $f\ne 0$
			- Append $\epsilon$-SPPF node numbered $f$ to $nodeSequence$
		- Call $z.\textrm{addChildren}(nodeSequence)$

In [ ]:
class RNGLRParser(RNGLRParser):
	    def reducer(self, i: int):
        '''
            The reducer, implemented based on pseudocode by Giorgios Robert Economopoulos
        '''
        v, X, m, f, y = self.reductions.pop()
        # print(f"[Reducer level ${i}] processing: ", v, X, m)
        # find the set of nodes that can be reached from v with lenght m-1
        paths = v.find_paths_with_length(max(0, m - 1))
        z: SPPFNode = None

        for path in paths:
            u = path[-1]
            k = u.label

            if m == 0:
                z = self.sppf.epsilon_sppf[f]
            else:
                c = u.level
                if (X, c) not in self.set_N:
                    z = self.sppf.create_node(X, c)
                    self.set_N[X, c] = z
                else:
                    z = self.set_N[(X, c)]
            for action in self.table[k][X]:
                action_obj = RNGLRParser.get_action(action)
                if action_obj[0] == 'p':
                    w = self.gss.find_node(action_obj[1], i)
                    if w is not None:
                        if u not in [x[0] for x in w.children]:
                            w.add_child(u, z)
                            if m != 0:
                                for action in self.table[action_obj[1]][self.input_str[i]]:
                                    action_obj = RNGLRParser.get_action(action)
                                    if action_obj[0] == 'r' and action_obj[2] != 0:
                                        # (u, B, t, f, z)
                                        self.add_reduction(u, action_obj[1], action_obj[2], action_obj[3], z)
                    else:
                        w = self.gss.create_node(action_obj[1], i)
                        w.add_child(u, z)
                        for action in self.table[action_obj[1]][self.input_str[i]]:
                            action_obj = RNGLRParser.get_action(action)
                            if action_obj[0] == 'p':
                                self.shifts.append((w, action_obj[1]))
                            if action_obj[0] == 'r':
                                t = action_obj[2]
                                if t == 0:
                                    self.add_reduction(w, action_obj[1], 0, action_obj[3], self.sppf.epsilon_sppf[0])
                                elif (m != 0):
                                    self.add_reduction(u, action_obj[1], t, action_obj[3], z)
            
            if (m != 0):
                # Add children
                node_seq = list(path[:-1])
                node_seq.reverse()
                if (m != 0):
                    node_seq.append(y)
                if (f != 0):
                    node_seq.append(self.sppf.epsilon_sppf[f])
                
                z.add_children(node_seq)

### References
[1] D. E. Knuth, “On the translation of languages from left to right,” _Information and Control_, vol. 8, no. 6, pp. 607–639, Dec. 1965, doi: [10.1016/S0019-9958(65)90426-2](https://doi.org/10.1016/S0019-9958\(65\)90426-2).

[2] G. R. Economopoulos, “Generalised LR parsing algorithms”. Retrieved from https://core.ac.uk/download/pdf/301667613.pdf

[3] M. Tomita, _Efficient Parsing for Natural Language_. Boston, MA: Springer US, 1986. doi: [10.1007/978-1-4757-1885-0](https://doi.org/10.1007/978-1-4757-1885-0).